In [1]:
# Imports with all of my tools
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os
import random
from sympy import mod_inverse
from PIL import Image as im

%matplotlib inline


def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    print(image.shape)
    return image

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
path = r"C:\Users\soule\OneDrive\Pictures\addie.jpg"
l = get_image(path)
img = im.open(path)
colors = img.getpixel((0, 1))
print(colors)
rows, cols = l.shape
# print(l)
data = im.fromarray(l.astype(np.uint8))
data.show()

(450, 600)
(30, 31, 35)


<IPython.core.display.Javascript object>

In [4]:
def Generate_Point(a, b, p):
    x = random.randint(0, p)
    print(x)
    y = np.arange(0, p)
    possibilites = []
    for i in range(len(y)):
        yval = int((x ** 3) % p)
        yval = int((yval + (a * x)) % p)
        yval = int((yval + b) % p)
        # yval = ((x ** 3) + (a * x) + b) % p
        # print(yval)
        # print(x[i],y[j],y[j]**2 % p, yval)
        # print(yval)
        testy = int((y[i] ** 2) % p)
        if testy == yval:
            possibilites.append(y[i])
            print(possibilites)
    if len(possibilites) == 0:
        return Generate_Point(a, b, p)
    else:
        yindex = random.randint(0, len(possibilites) - 1)
        return x, possibilites[yindex]

<IPython.core.display.Javascript object>

In [5]:
arr = []
for i in range(rows):
    for j in range(cols):
        arr.append(l[i][j])
print(arr[0], arr[1], arr[2])
print(l)

30 31 33
[[ 30  31  33 ...  71  79  95]
 [ 31  32  34 ...  76  74  99]
 [ 31  32  34 ... 105  87 113]
 ...
 [ 90  88  85 ... 116  98  91]
 [ 87  89  78 ... 111 120 101]
 [ 66  63  48 ... 110 129 101]]


<IPython.core.display.Javascript object>

In [6]:
totalPixs = rows * cols
print(totalPixs)

270000


<IPython.core.display.Javascript object>

In [7]:
imagePointsArray = np.reshape(arr, (totalPixs // 2, 2), order="C")
print(imagePointsArray)

[[ 30  31]
 [ 33  36]
 [ 39  41]
 ...
 [ 85  92]
 [ 97 110]
 [129 101]]


<IPython.core.display.Javascript object>

In [8]:
p = 1046527  # 16127
a = -5000
b = 76193

<IPython.core.display.Javascript object>

In [9]:
# G = random point
Gx, Gy = Generate_Point(a, b, p)
print(Gx, Gy)

359938
[138771]
[138771, 953680]
359938 138771


<IPython.core.display.Javascript object>

In [10]:
# Amanda's alpha and point A
private_a = 741
private_Ax, private_Ay = Generate_Point(a, b, p)
print(private_Ax, private_Ay)

417021
463138
[642650]
463138 642650


<IPython.core.display.Javascript object>

In [11]:
# Bill's beta and point B
private_b = 9812
private_Bx, private_By = Generate_Point(a, b, p)
print(private_Bx, private_By)

542172
[196612]
542172 196612


<IPython.core.display.Javascript object>

In [12]:
x1t, y1t = Gx, Gy
((x1t ** 3) % p + (a * x1t) % p + b) % p

165162

<IPython.core.display.Javascript object>

In [13]:
(y1t ** 2) % p

165162

<IPython.core.display.Javascript object>

In [14]:
def point_addition(x1, y1, x2, y2, p, a):
    z = 0
    if x1 == z and y1 == z:
        return x2, y2
    if x2 == z and y2 == z:
        return x1, y1
    if x1 == x2:
        if x1 == x2 and y1 == -y2 % p:
            return 0, 0
        else:
            den = int(mod_inverse(2 * y1, p) % p)
            m = int((3 * (x1 ** 2)) % p)
            m = int((m + a) % p)
            m = int((m * den) % p)
            m = int((((3 * (x1 ** 2)) % p + a) * (den)) % p)
            # m = (((3*(x1**2))%p + a) * (2*))
            # m = ((3*pow(x1, 2, p) + a)*pow(2*y1, p-2, p)) % p
            # print(m)
    else:
        den = int(mod_inverse(x1 - x2, p) % p)
        m = int((y1 - y2) % p)
        m = int((m * den) % p)
        # m = int(((y1 - y2)*den)%p)
        # print(m)
    x = int((m ** 2) % p)
    x = int((x - x1 - x2) % p)
    # x = int(((m**2)%p - x1 - x2) % p)
    # print(x)
    y = int((x1 - x) % p)
    y = int((y * m) % p)
    y = int((y - y1) % p)
    # y = int((m*(x1 - x)%p - y1) % p)
    # print(y)
    return int(x), int(y)


def point_subtraction(x1, y1, x2, y2, p, a):
    y2 = int(-(y2))
    y2 = int(y2 % p)
    return point_addition(x1, y1, x2, y2, p, a)


def point_multiplication(x, y, k, p, a):
    xnew, ynew = int(x), int(y)
    for i in range(k - 1):
        xnew, ynew = point_addition(x, y, xnew, ynew, p, a)
        # print(xnew,ynew)
    return int(xnew), int(ynew)

<IPython.core.display.Javascript object>

def point_addition(x1, y1, x2, y2, p, a):
    z = 0
    if x1 == z and y1 == z:
        return x2, y2
    if x2 == z and y2 == z:
        return x1, y1
    if x1 == x2:
        if x1 == x2 and y1 == -y2 % p:
            return 0, 0
        else:
            den = mod_inverse(2 * y1, p)
            m = (((3 * (x1 ** 2)) % p + a) * (den)) % p
    else:
        den = mod_inverse(x1 - x2, p)
        m = (y1 - y2) * den % p

    x = ((m ** 2) % p - x1 - x2) % p

    y = (m * (x1 - x) % p - y1) % p

    return x, y


def point_subtraction(x1, y1, x2, y2, p, a):
    y2 = -(y2)
    y2 = y2 % p
    return point_addition(x1, y1, x2, y2, p, a)


def point_multiplication(x, y, k, p, a):
    xnew, ynew = x, y
    for i in range(k - 1):
        xnew, ynew = point_addition(x, y, xnew, ynew, p, a)
        # print(xnew,ynew)
    return xnew, ynew

In [15]:
# G+A
G_plus_Ax, G_plus_Ay = point_addition(Gx, Gy, private_Ax, private_Ay, p, a)

<IPython.core.display.Javascript object>

In [16]:
# A_1 = alpha*(G+A)
A1x, A1y = point_multiplication(G_plus_Ax, G_plus_Ay, private_a, p, a)

<IPython.core.display.Javascript object>

In [17]:
# A_2 = alpha * A
A2x, A2y = point_multiplication(private_Ax, private_Ay, private_a, p, a)

<IPython.core.display.Javascript object>

In [18]:
# G+B
G_plus_Bx, G_plus_By = point_addition(Gx, Gy, private_Bx, private_By, p, a)

<IPython.core.display.Javascript object>

In [19]:
# B_1 = beta * (G+B)
B1x, B1y = point_multiplication(G_plus_Bx, G_plus_By, private_b, p, a)

<IPython.core.display.Javascript object>

In [20]:
# B_2 = beta * B
B2x, B2y = point_multiplication(private_Bx, private_By, private_b, p, a)

<IPython.core.display.Javascript object>

In [21]:
# A_B = alpha * B_2
A_Bx, A_By = point_multiplication(B2x, B2y, private_a, p, a)

<IPython.core.display.Javascript object>

In [22]:
# B_A = beta * A_2
B_Ax, B_Ay = point_multiplication(A2x, A2y, private_b, p, a)

<IPython.core.display.Javascript object>

In [23]:
# (beta + 1) * A_1
b_plus_one_A1 = point_multiplication(A1x, A1y, (private_b + 1), p, a)

# (beta + 1)*A_1 - A_2
b_plus_one_A1_minus_A2 = point_subtraction(
    b_plus_one_A1[0], b_plus_one_A1[1], A2x, A2y, p, a
)

# (beta + 1)*A_1 - A_2 + A_B
added_to_M = point_addition(
    b_plus_one_A1_minus_A2[0], b_plus_one_A1_minus_A2[1], A_Bx, A_By, p, a
)

<IPython.core.display.Javascript object>

In [24]:
coordinates, variables = imagePointsArray.shape
encrypted_array = []
remainder_array = []
print(coordinates)
r = coordinates
for i in range(coordinates):
    # M + (beta + 1)*A_1 - A_2 + A_B
    EP = point_addition(
        imagePointsArray[i][0],
        imagePointsArray[i][1],
        added_to_M[0],
        added_to_M[1],
        p,
        a,
    )
    encrypted_array.append(EP[0])
    encrypted_array.append(EP[1])
print(len(encrypted_array))

135000
270000


<IPython.core.display.Javascript object>

final_encrypted_array = []
for i in range(len(encrypted_array)):
    scaled_value = encrypted_array[i] % 256
    divisor = encrypted_array[i] // 256
    final_encrypted_array.append(scaled_value)
    final_encrypted_array.append(divisor)

In [25]:
encryption_arr = np.array(encrypted_array, dtype=int)
encryption_arr = np.reshape(encryption_arr, (rows, cols), order="C")
print(encryption_arr)
e2 = encryption_arr % 256
print(e2)

[[821614 743959 600709 ... 517247 972108 371749]
 [297532 263830 337571 ... 582780 189164 138579]
 [297532 263830 337571 ... 913321 323851 478491]
 ...
 [338700 618111 508261 ... 155429 200855 240486]
 [308426 327714 567418 ...  46959 622000 288753]
 [586885 134156 134262 ... 584580 320240 792351]]
[[110  23 133 ... 127  76  37]
 [ 60 150 163 ... 124 236  83]
 [ 60 150 163 ... 169  11  27]
 ...
 [ 12 127 101 ...  37 151 102]
 [202  34 122 ... 111 176 241]
 [133  12 118 ... 132 240  31]]


<IPython.core.display.Javascript object>

In [26]:
print(459362 % 256)

98


<IPython.core.display.Javascript object>

In [27]:
e_data = im.fromarray(e2.astype(np.uint8))
e_data.show()

<IPython.core.display.Javascript object>

# Decryption
data = encryption_arr
x, y = data.shape
print(x, y)
print(data.shape)
# print(data)

In [28]:
arr = []
for i in range(rows):
    for j in range(cols):
        arr.append(encryption_arr[i][j])
print(len(arr))
print(arr[0], arr[1], arr[2], arr[3])
# print(arr)

270000
821614 743959 600709 23686


<IPython.core.display.Javascript object>

apply_divisor = np.reshape(encryption_arr, (len(arr) // 2, 2), order="C")
applied_data = []
for i in range(len(arr) // 2):
    newValue = apply_divisor[i][1] * 256 + apply_divisor[i][0]
    applied_data.append(newValue)
print(len(applied_data))

x = x // 2

In [29]:
# newTotal = x * y
EmPointsArray = np.reshape(arr, (totalPixs // 2, 2), order="C")
print(EmPointsArray)

[[821614 743959]
 [600709  23686]
 [329123 896413]
 ...
 [614971 912706]
 [151065 584580]
 [320240 792351]]


<IPython.core.display.Javascript object>

In [30]:
# G + B_1
G_plus_B1 = point_addition(Gx, Gy, B1x, B1y, p, a)

# alpha * (G+B_1) = alpha*G + alpha*B_1
private_a_G_plus_B1 = point_multiplication(G_plus_B1[0], G_plus_B1[1], private_a, p, a)

# alpha * (G+B_1) + B_A
private_a_G_plus_B1_plus_BA = point_addition(
    private_a_G_plus_B1[0], private_a_G_plus_B1[1], B_Ax, B_Ay, p, a
)

<IPython.core.display.Javascript object>

In [31]:
coordinates, variables = EmPointsArray.shape
decrypted_array = []
print(coordinates)
r=coordinates
for i in range(coordinates):
    #C_E - (alpha * (G+B_1) + B_A)
    DP = point_subtraction(EmPointsArray[i][0],EmPointsArray[i][1],private_a_G_plus_B1_plus_BA[0],private_a_G_plus_B1_plus_BA[1],p,a)
    decrypted_array.append(DP[0]%256)
    decrypted_array.append(DP[1]%256)


135000


<IPython.core.display.Javascript object>

In [32]:
decryption_arr = np.array(decrypted_array, dtype=int)
decryption_arr = np.reshape(decryption_arr, (rows, cols), order="C")
decryption_arr

array([[ 31,   8, 210, ...,  95, 136, 124],
       [ 77, 130,  13, ..., 197, 188, 237],
       [ 77, 130,  13, ..., 162,  70, 219],
       ...,
       [130, 168, 110, ...,  54, 100, 140],
       [ 14, 121,  22, ..., 183, 248,  87],
       [165, 157, 150, ..., 225, 199,  67]])

<IPython.core.display.Javascript object>

In [33]:
d_data = im.fromarray(decryption_arr.astype(np.uint8))
d_data.show()

<IPython.core.display.Javascript object>